In [18]:
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import itertools

from config import shuffled_csv, path_exps
from NN import NN_model, Sigmoid, MSE, L2_reg, L1_reg, ReLU
from NN.utility import batch_train, batch_out, Model_Wrapper
from LBFGS import LBFGS
from LevelMethod import LevelMethod
from testing import multi_run


In [7]:
data = pd.read_csv(shuffled_csv, index_col=0).to_numpy()
data = data[:100, :]
n_samples = data.shape[0]
X_data = data[:, :10]
Y_data = data[:, 10:]

Y_scaler = StandardScaler()
Y_scaled = Y_scaler.fit_transform(Y_data)

# np.random.seed(11)
model = NN_model([10, 20, 20, 2], ReLU, MSE)
model.init_weights()
reg_loss = L1_reg(1e-4)

# logging.basicConfig(level="INFO")
f = Model_Wrapper(model, X_data, Y_scaled, reg_loss)
x = model.Weights

##########################################
lambda_ = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
bounds = [1]
HPs = itertools.product(lambda_, bounds)

experiments_csv = path_exps / Path("level_grid_search.csv")
# remove HP combinations already explored
if experiments_csv.exists():
    df = pd.read_csv(experiments_csv)
    HPs_new = []
    for lambda_, bounds in HPs:
        if ((df['lambda_'] == lambda_) & (df['bounds'] == bounds)).any():
            continue
        else:
            HPs_new.append((lambda_, bounds))
    HPs = HPs_new

for lambda_, bounds in HPs:
    print(f'Testing lambda: {lambda_}, bounds: {bounds}', end='\n')
    solver = LevelMethod(lambda_=lambda_, bounds=bounds, max_iter=100000, verbose=False)
    results = multi_run(solver, f, n=3)

    log_columns = ["lambda_", "bounds", "final_fv_m", "final_fv_std", "f_evals_m", "f_evals_std", "seconds_m", "seconds_std", "n_failures"]
    log_data = [lambda_, bounds, results[0], results[1], results[2], results[3], results[4], results[5], results[6]]
    df = pd.DataFrame([log_data], columns=log_columns)

    if experiments_csv.exists():
        df.to_csv(experiments_csv, mode="a", header=False,index=False)
    else:
        if not os.path.exists(path_exps):
            os.mkdir(path_exps)
        df.to_csv(experiments_csv,index=False)
    print('\t Finished')


Testing lambda: 0.1, bounds: 1
LevelMethodException Warning: Negative gap -0.7290820725613685
	 Finished


In [30]:
experiments_csv = path_exps / Path("level_grid_search.csv")
df = pd.read_csv(experiments_csv)

x = df['lambda_']

plt.plot(x,df['final_fv_m'], label='min f')
plt.xlabel('lambda')
plt.ylabel('min f')
plt.savefig('minf.png')
plt.clf()

plt.plot(x,df['f_evals_m'], label='f evals')
plt.xlabel('lambda')
plt.ylabel('f evals')
plt.savefig('f evals.png')
plt.clf()

plt.plot(x,df['seconds_m'], label='seconds')
plt.xlabel('lambda')
plt.ylabel('seconds')
plt.savefig('seconds.png')
plt.clf()

df.sort_values(by=['final_fv_m'])

,lambda_,bounds,final_fv_m,final_fv_std,f_evals_m,f_evals_std,seconds_m,seconds_std,n_failures
8,0.9,1,0.011775,0.000273,911.333333,13.888444,3392.401042,182.779232,0
7,0.8,1,0.038847,0.024011,832.666667,142.230642,2959.723958,1057.441325,0
5,0.6,1,1.138086,0.772114,505.666667,251.555605,1125.447917,1186.488504,0
6,0.7,1,2.513411,3.336312,694.666667,119.402215,1880.317708,758.271399,0
4,0.5,1,2.907702,2.537906,433.666667,34.412530,565.822917,123.785897,0
3,0.4,1,20.816098,12.046484,254.333333,28.871362,186.234375,25.078792,0
0,0.1,1,33.922419,27.212647,492.333333,124.097632,904.109375,478.838404,0
2,0.3,1,62.714657,50.602665,177.000000,109.584062,122.968750,97.837367,0
1,0.2,1,132.242422,47.403336,145.000000,86.733308,92.088542,68.458265,0


<Figure size 432x288 with 0 Axes>